<a href="https://colab.research.google.com/github/gp201/BERT/blob/master/create_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import re
import pandas as pd
from csv import writer

In [0]:
# tags from corpus
tagged_corpus = pd.read_csv("corpus_tags.tsv", sep="\t", names=["pmid", "start", "end", "word", "type", "umls_id"])
tagged_corpus = tagged_corpus.dropna()
tagged_corpus.iloc[610:640]

,pmid,start,end,word,type,umls_id
610,26867927,0,23,Radiofrequency ablation,T061,C0850292
611,26867927,27,59,posteroseptal accessory pathways,T023,C0447025
612,26867927,60,75,associated with,T080,C0332281
613,26867927,76,102,coronary sinus diverticula,"T019,T047",C3163894
614,26867927,103,135,Posteroseptal accessory pathways,T023,C0447025
615,26867927,143,158,associated with,T080,C0332281
616,26867927,161,193,coronary sinus (CS) diverticulum,"T019,T047",C3163894
617,26867927,227,251,clinical characteristics,T201,C0683325
618,26867927,253,260,mapping,T052,C1283195
619,26867927,265,291,ablation of these pathways,T061,C3275043


In [0]:
classes = []
with open("vocab_class.txt") as ftags:
    classes = [i.strip() for i in ftags.readlines()]
    ftags.close()
classes

['T047', 'T005', 'T007', 'T016']

In [0]:
sentence_count = 1
def get_words_tags(text, pmid,start=0):
    words = []
    tags = []  
    row_cont = []
    typ = "O"
    global sentence_count
    df = tagged_corpus[(tagged_corpus["pmid"]==int(pmid)) & (tagged_corpus["end"] <= start + len(text)) & (tagged_corpus["start"] >= start)]
    indices = df.index.values
    #print(indices)
    for word in text.split():
        for i in indices:
            #print('jj',i)
            text2 = df.loc[i]["word"]
            #print('$$',text2.split())
            if word in text2.split() and set(df.loc[i]["type"].split(',')).intersection(classes):
                typ = df.loc[i]["type"]
                break
            else:
                typ = "O"
            #print("#",word,"#",typ)
        row_cont.append(["Sentence: " + str(sentence_count),word,typ])
    sentence_count = sentence_count + 1
    return row_cont



In [0]:
title_regex = re.compile("^\d+\|t\|")
abstract_regex = re.compile("^\d+\|a\|")


fr = open("corpus_pubtator.txt") #corpus file should be placed here

fw = open('dataset.csv','a')
fw_writer = writer(fw)
first_row = ["Sentence #","Word","Tag"]
fw_writer.writerow(first_row)
fw.close()

In [0]:
def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        for row in list_of_elem:
          csv_writer.writerow(row)
        blank = ['','','']
        csv_writer.writerow(blank)

In [0]:
abstract_count = 0
for i in fr.readlines():
    row_content = []
    line = i.strip()
    title_search = title_regex.search(line)
    abstract_search = abstract_regex.search(line)
    if title_search:
        title_line = title_regex.sub('',line)
    elif abstract_search:
        abstract = abstract_regex.sub('',line)
        pmid = int(abstract_search.group(0)[:-3])
        row_content = get_words_tags(title_line,pmid,0)
        append_list_as_row('dataset.csv', row_content)
        title_size = len(title_line) + 1                     # 1 for \n        
        for sentence in re.split("\.", abstract):
            if len(sentence) == 0:
                continue
            _size = abstract.index(sentence) + title_size
            row_content = get_words_tags(sentence, pmid,_size)
            append_list_as_row('dataset.csv', row_content)
        abstract_count += 1
        if abstract_count%10 == 0:
            print("PMID {} written to file. Abstract Count: {}".format(pmid, abstract_count))
fr.close()
print("Completed")